In [60]:
import numpy as np

In [61]:
# импорты для интерактива
from ipywidgets import interact, fixed
import ipywidgets as widgets

In [62]:
# вектор свободных членов (start_i, start_j) -> (end_i, end_j) -> value
free_terms = {
    (0, 1): {  # координата A
        (4, 1): 10,  # координата A' -> значение
    },
    (0, 3): {  # координата B
        (2, 3): 5,  # координата B' -> значение
    }
}

In [63]:
# задание интерактива и ограничений для значений
@interact(
    height=widgets.IntSlider(min=1, max=10, step=1, value=5),
    width=widgets.IntSlider(min=1, max=10, step=1, value=5),
    gamma=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.9),
    borders_penalty=widgets.IntSlider(min=-10, max=0, step=1, value=-1),
    free_terms=fixed(free_terms),
)
def solve_problem(height, width, gamma, borders_penalty, free_terms):

    A = np.zeros((height * width, height * width))
    b = np.zeros(height * width)  # Коэффициент при V(s)
    
    for row in range(height):
        for col in range(width):
            idx_flatten = row * width + col  # одномерное представление координат
            A[idx_flatten, idx_flatten] = -1

            # соседние клетки
            next_idxes = ((row - 1, col), (row, col - 1), (row + 1, col), (row, col + 1))
    
            for i, j in next_idxes:
                if 0 <= j <= width - 1 and 0 <= i <= height - 1:
                    next_idx_flatten = i * width + j  # одномерное представление координат
                    A[idx_flatten, next_idx_flatten] += gamma / len(next_idxes)  # дисконтирование
                else:
                    # штрафуем за выход за пределы поля
                    b[idx_flatten] -= borders_penalty / len(next_idxes)

    # обрабатываем все свободные члены
    for (start_i, start_j), dict_value in free_terms.items():
        for (next_i, next_j), value in dict_value.items():
            idx_flatten = start_i * width + start_j  # одномерное представление координат
            next_idx_flatten = next_i * width + next_j  # одномерное представление координат
            A[idx_flatten, next_idx_flatten] = gamma
            b[idx_flatten] = -value

    # решение СЛАУ и возврат решения в форме матрицы
    return np.linalg.solve(A, b).reshape((height, width))

interactive(children=(IntSlider(value=5, description='height', max=10, min=1), IntSlider(value=5, description=…